In [22]:
from torchvision.datasets import MNIST
import torchvision
import numpy as np
import torch.nn.functional as F
from torchvision.transforms import ToPILImage
to_img = ToPILImage()
from PIL import Image
from torch.autograd import Variable
import torch
import torch.nn as nn

In [19]:
transform=torchvision.transforms.Compose([ torchvision.transforms.ToTensor()])

train_dataset=MNIST(root="~/torch_datasets", train=True, transform=transform, download=True)

test_dataset=MNIST(root="~/torch_datasets", train=False, transform=transform, download=True)


train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True)

test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
class MNISTUpscaler(nn.Module):
    def __init__(self):
        super(MNISTUpscaler,self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5),
            nn.ReLU(True),
            nn.Conv2d(32,32,kernel_size=5),
            nn.ReLU(True), 
            nn.Conv2d(32, 32, kernel_size=4),
            nn.ReLU(True),
            nn.Conv2d(32, 32, kernel_size=3))
        self.decoder = nn.Sequential(  
            nn.ConvTranspose2d(32, 32, kernel_size=4),
            nn.ReLU(True),
            nn.ConvTranspose2d(32,32,kernel_size=3, stride=2),
            nn.ReLU(True),
            nn.ConvTranspose2d(32,32,kernel_size=4, stride=2),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 32, kernel_size=5),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 1, kernel_size=5))
        
        

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [12]:
model = MNISTUpscaler()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
num_epochs=5

In [23]:


for epoch in range(num_epochs):
    for data in train_loader:
        model.train()
        img, _= data
        inputt=F.avg_pool2d(img, kernel_size=2)
        # ===================forward=====================
        output = model(inputt)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.item()))





epoch [1/5], loss:0.0165
epoch [2/5], loss:0.0099
epoch [3/5], loss:0.0083
epoch [4/5], loss:0.0075
epoch [5/5], loss:0.0064


In [25]:
loss=0
num=0
for data in test_loader:
    model.eval()
    img, _=data
    inputt=F.avg_pool2d(img, kernel_size=2)
    output=model(inputt)
    
    loss+=criterion(output, img)
    num+=1
    
loss/=float(num)    
print('test_loss:{:.4f}'.format(loss))


torch.save(model.state_dict(), './MNISTUpscaler.pth')

test_loss:0.0060
